In [1]:
import pandas as pd
import numpy as np

import os

print(f'Current working directory: {os.getcwd()}')
os.chdir('../')
print(f'Changed working directory: {os.getcwd()}')

Current working directory: /content
Changed working directory: /


### load data

In [2]:
data = pd.read_csv('/content/sample_data/cleaned_Brouwer_2021.csv')
print(data.shape)
data.head()

(20870, 4)


,solute,solvent,T,log_gamma
0,C,CCCCCCCCCCCCCCCC,40.0,-0.261365
1,C,CCCCCCCCCCCCCCCC,70.0,-0.287682
2,C,CCCCCCCCCCCCCCCC,90.0,-0.301105
3,CC,CCCCCCCCCCCCCCCC,40.0,-0.235722
4,CC,CCCCCCCCCCCCCCCC,70.0,-0.248461


## Split Data

In [3]:
import numpy as np
import pandas as pd

class SmilesData:
    def __init__(self, data):
        self.data = data

    def get_split(self, train_ratio=0.8, seed=None):
        n = len(self.data)
        indices = np.arange(n)
        if seed is not None:
            np.random.seed(seed)
        np.random.shuffle(indices)
        train_size = int(train_ratio * n)
        train_indices = indices[:train_size]
        test_indices = indices[train_size:]
        train_data = self.data.iloc[train_indices].reset_index(drop=True)
        test_data = self.data.iloc[test_indices].reset_index(drop=True)
        return train_data, test_data

train_data, test_data = SmilesData(data).get_split(seed=2024)
print(f'Shape of train data: {train_data.shape}')
print(f'Shape of test data: {test_data.shape}')

Shape of train data: (16696, 4)
Shape of test data: (4174, 4)


### tokenizer data

In [4]:
from transformers import AutoTokenizer
from transformers import AutoConfig
from transformers import AutoModelForSequenceClassification
from transformers import TrainingArguments

# load in the tokenizer
model_name = "seyonec/PubChem10M_SMILES_BPE_450k" #"DeepChem/ChemBERTa-77M-MTR" #'seyonec/PubChem10M_SMILES_BPE_450k'
tokenizer = AutoTokenizer.from_pretrained(model_name)

tokenizer_config.json:   0%|          | 0.00/62.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/515 [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/165k [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/101k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/772 [00:00<?, ?B/s]

In [5]:
import torch
from torch.utils.data import Dataset
class Input(Dataset):
    def __init__(self, data, tokenizer, column_names = ['solute', 'solvent', 'T', 'log_gamma']):
        '''
        data: pandas dataframe with columns "solute", "solvent", "T", "log_gamma
        tokenizer: tokenizer to use'''

        self.data = data
        self.tokenizer = tokenizer
        self.column_names = column_names
        self.max_length = self.max_len(column_names[0]) + self.max_len(column_names[1]) + 3 # The total character length including [CLS], [SEP], and [PAD]

    def __len__(self):
        '''Returns the length of the dataset'''
        return len(self.data)

    def max_len(self, idx):
        """
        Returns the maximum length of the input sequence
        """
        return max([len(x) for x in self.data[idx]])

    def __getitem__(self, idx):
        """
        Returns the input data as a dictionary with keys "solute", "solvent", "input_ids", "attention_mask", "Temp", and "label"
        """
        solute = self.data.iloc[idx][self.column_names[0]]
        solvent = self.data.iloc[idx][self.column_names[0]]
        inputs = self.tokenizer(solute, solvent, return_tensors="pt", padding='max_length', truncation=True,
                                max_length=self.max_length)

        sample = {
                    'solute': solute,
                    'solvent': solvent,
                    'input_ids': inputs["input_ids"].squeeze(0),
                    'attention_mask': inputs["attention_mask"].squeeze(0),
                    'Temp': torch.tensor(self.data.iloc[idx][self.column_names[2]], dtype=torch.float).unsqueeze(0),
                    'label': torch.tensor(self.data.iloc[idx][self.column_names[3]], dtype=torch.float).unsqueeze(0)
                    }
        return sample
# create the train and test datasets
TrainData = Input(train_data, tokenizer)
TestData = Input(test_data, tokenizer)
# Shape
print(f'Length of train data: {len(TrainData)}')
print(f'Length of test data: {len(TestData)}')

Length of train data: 16696
Length of test data: 4174


In [6]:
TrainData[0]

{'solute': 'CC(C)CC(C)(C)C',
 'solvent': 'CC(C)CC(C)(C)C',
 'input_ids': tensor([  0, 262,  12,  39,  13, 262,  12,  39, 274,  39,  13,  39,   2,   2,
         262,  12,  39,  13, 262,  12,  39, 274,  39,  13,  39,   2,   1,   1,
           1,   1,   1,   1,   1,   1,   1,   1,   1,   1,   1,   1,   1,   1,
           1,   1,   1,   1,   1,   1,   1,   1,   1,   1,   1,   1,   1,   1,
           1,   1,   1,   1,   1,   1,   1,   1,   1,   1,   1,   1,   1,   1,
           1,   1,   1,   1,   1,   1,   1,   1,   1,   1,   1,   1,   1,   1,
           1,   1,   1,   1,   1,   1,   1,   1,   1,   1,   1,   1,   1,   1,
           1,   1,   1,   1,   1,   1,   1,   1,   1,   1,   1,   1,   1,   1,
           1,   1,   1,   1,   1,   1,   1,   1,   1,   1,   1,   1,   1,   1,
           1,   1,   1,   1,   1,   1,   1,   1,   1,   1]),
 'attention_mask': tensor([1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
         1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 

## Model

In [ ]:
# import torch
# import torch.nn as nn
# import torch.nn.functional as F
# from transformers.models.roberta.modeling_roberta import RobertaModel, RobertaClassificationHead

# class SmilesTransformerModel(nn.Module):
#     def __init__(self,
#                  model_name = "seyonec/PubChem10M_SMILES_BPE_450k", # model name
#                  n_classes=1):
#         super(SmilesTransformerModel, self).__init__()
#         # pretrained model
#         config = AutoConfig.from_pretrained(model_name,return_dict = False) # configuration
#         self.transformer = RobertaModel.from_pretrained(model_name, config=config,add_pooling_layer=False) # load in the model
#         self.drop = nn.Dropout(0.3)
#         self.fc1 = nn.Linear(self.transformer.config.hidden_size, self.transformer.config.hidden_size)
#         self.fc2 = nn.Linear(self.transformer.config.hidden_size, self.transformer.config.hidden_size)
#         self.fc3 = nn.Linear(self.transformer.config.hidden_size, n_classes)


#     def forward(self, input_ids: torch.Tensor, attention_mask: torch.Tensor = None, temperature: torch.Tensor = None, labels: torch.Tensor = None):
#         """
#         :param input_ids: SMILES encodings
#         :param attention_mask: attention mask (1 for non-padding token and 0 for padding)
#         :param temperature: temperature
#         """

#         outputs = self.transformer(input_ids = input_ids,
#                                     attention_mask = attention_mask)
#         sequence_output = outputs[0]
#         #print('transformer_outputs', sequence_output.shape)

#         drop_out = self.drop(sequence_output)
#         # fc1 = F.relu(self.fc1(drop_out))
#         # fc2 = F.relu(self.fc2(fc1))
#         #print('drop_out', drop_out.shape)
#         output = self.fc3(drop_out)
#         return output

In [7]:
import torch
import torch.nn as nn
import torch.nn.functional as F
from transformers.models.roberta.modeling_roberta import RobertaModel

class ClassificationHead(nn.Module):
    """Head for sentence-level classification tasks."""

    def __init__(self, config, num_labels=1):
        super().__init__()
        self.dense = nn.Linear(config.hidden_size, config.hidden_size)
        classifier_dropout = (
            config.classifier_dropout if config.classifier_dropout is not None else config.hidden_dropout_prob
        )
        self.dropout = nn.Dropout(classifier_dropout)
        self.out_proj = nn.Linear(config.hidden_size, num_labels)

    def forward(self, features, **kwargs):
        x = features[:, 0, :]  # take <s> token (equiv. to [CLS])
        x = self.dropout(x)
        x = self.dense(x)
        x = torch.tanh(x)
        x = self.dropout(x)
        x = self.out_proj(x)
        return x

class SmilesTransformerModel(nn.Module):
    def __init__(self,
                 model_name = "seyonec/PubChem10M_SMILES_BPE_450k", # model name
                 n_classes=1):
        super(SmilesTransformerModel, self).__init__()
        # pretrained model
        config = AutoConfig.from_pretrained(model_name) # configuration
        self.transformer = RobertaModel.from_pretrained(model_name, config=config,add_pooling_layer=False) # load in the model
        self.classifier = ClassificationHead(config, n_classes)


    def forward(self, input_ids: torch.Tensor, attention_mask: torch.Tensor = None, temperature: torch.Tensor = None, labels: torch.Tensor = None):
        """
        :param input_ids: SMILES encodings
        :param attention_mask: attention mask (1 for non-padding token and 0 for padding)
        :param temperature: temperature
        """

        outputs = self.transformer(input_ids = input_ids,
                                    attention_mask = attention_mask)
        sequence_output = outputs[0]
        #print('transformer_outputs', sequence_output.shape)
        output = self.classifier(sequence_output)
        return output

In [ ]:
# batch_size = 128
# loss_fn = nn.MSELoss()
# smilesmodel = SmilesTransformerModel()
# train_loader = torch.utils.data.DataLoader(TrainData, batch_size=batch_size, shuffle=True, pin_memory=True)
# for batch in train_loader:
#     print('input_ids: ', batch['input_ids'].shape, 'attention_mask: ', batch['attention_mask'].shape)
#     print('Temp: ', batch['Temp'].shape, 'label: ', batch['label'].shape)
#     out = smilesmodel(input_ids=batch['input_ids'], attention_mask=batch['attention_mask'], temperature=batch['Temp'], labels=batch['label'])
#     print('out', out.shape)
#     print('loss: ', loss_fn(out, batch['label']))
#     break

In [ ]:
import torch.nn as nn
from torch.utils.data import DataLoader
from torch.utils.data.sampler import SubsetRandomSampler
from sklearn.model_selection import KFold


class ModelTrainer(nn.Module):

    """
        ************** Train/Test the model using cross validation **************
        seed: seed for random number generator
        epochs: number of epochs to train
        lr: learning rate
        train: flag whether to train the model
        log_interval: how many batches to wait before logging training status
        model: takes input_ids: str, attention_mask: str, classification: bool

    """

    def __init__(self, seed = 2023, lr=2e-3, epochs=10):
        super(ModelTrainer, self).__init__()
        self.seed = seed
        self.epochs = epochs
        self.lr = lr
        self.device = torch.device("cuda" if torch.cuda.is_available() else ("mps" if torch.backends.mps.is_available() else "cpu"))
        self.model = SmilesTransformerModel().to(self.device)

        self.optimizer = torch.optim.AdamW(self.model.parameters(), lr=self.lr)

        self.loss_func = nn.MSELoss()

    def validate(self, val_loader, model, device, loss_func):
        """Evaluate the network on the entire validation (part of training data) set."""

        val_loss, val_accuracy = 0, 0

        model.eval()
        with torch.no_grad():

            for data in val_loader:
                # get the inputs
                input_ids = data['input_ids'].to(device)
                input_mask = data['attention_mask'].to(device)
                labels = data['label'].to(device)
                # forward pass
                outputs = model(input_ids=input_ids, attention_mask=input_mask)
                # loss and accuracy
                loss = loss_func(outputs, labels)
                val_loss += loss.sum().item() * input_ids.size(0)


        return val_loss, val_accuracy

    def test(self, test_loader, model, loss_func, device):
        """Evaluate the network on the entire test set and calculate AUC."""

        model.eval()

        test_loss, test_accuracy = 0, 0

        with torch.no_grad():
            for data in test_loader:
                # get the inputs
                input_ids = data['input_ids'].to(device)
                input_mask = data['attention_mask'].to(device)
                labels = data['label'].to(device)

                # forward pass
                outputs = model(input_ids=input_ids, attention_mask=input_mask)

                # loss and accuracy
                loss = loss_func(input=outputs, target=labels)
                test_loss += loss.sum().item() * input_ids.size(0)

        return test_loss, test_accuracy


    def train(self, model, train_loader, loss_func, optimizer, device):
        """Train the network on the training set."""
        train_loss, train_accuracy = 0, 0

        model.train()
        scheduler = torch.optim.lr_scheduler.OneCycleLR(optimizer, max_lr=0.01, steps_per_epoch=len(train_loader), epochs=self.epochs, anneal_strategy='linear')

        for data in train_loader:

            # get the inputs
            input_ids = data['input_ids'].to(device)   # amino acid index numbers
            input_mask = data['attention_mask'].to(device) # attention mask (1 for non-padding token and 0 for padding)
            labels = data['label'].to(device) # True for classification task
            # forward pass
            outputs = self.model(input_ids = input_ids, attention_mask = input_mask)
            # loss and backward pass
            loss = loss_func(input=outputs, target=labels)
            loss.mean().backward()
            optimizer.step()
            scheduler.step()
            optimizer.zero_grad()
            # loss and accuracy
            train_loss += loss.sum().item() * input_ids.size(0)

        return train_loss, train_accuracy

    def execute_run(self, train_loader, test_loader, fold = 3, batch_size = 32):
        '''Train, Test and Validate the network on the training set using cross validation.'''

        print(f"Training on: {self.device}")

        torch.manual_seed(self.seed) # set the seed for generating random numbers

        if torch.cuda.is_available():
            torch.cuda.manual_seed(self.seed)

        # split data for K-fold cross validation to avoid overfitting
        self.fold = fold
        indices = list(range(len(train_loader.dataset)))
        kf = KFold(n_splits=self.fold, shuffle=True)

        for cv_index, (train_indices, valid_indices) in enumerate(kf.split(indices)):

            train_sampler = SubsetRandomSampler(train_indices)
            valid_sampler = SubsetRandomSampler(valid_indices)

            train_loader = DataLoader(train_loader.dataset, batch_size=batch_size,
                                                       sampler=train_sampler,
                                                       shuffle=False, pin_memory=True)
            val_loader = DataLoader(train_loader.dataset, batch_size=batch_size,
                                                     sampler=valid_sampler,
                                                     shuffle=False, pin_memory=True)

            print("CV: {}".format(cv_index))

            self.history = {'train_loss': [], 'val_loss': []}

            for epoch in range(0, self.epochs + 1):
                # Training
                epoch_train_loss, epoch_train_accuracy = self.train(model=self.model,
                                                                    train_loader=train_loader, loss_func=self.loss_func,
                                                                    optimizer=self.optimizer, device=self.device)
                # Validation
                epoch_val_loss, epoch_val_accuracy = self.validate(val_loader=val_loader,
                                                                    model=self.model, loss_func=self.loss_func,
                                                                    device=self.device)
                #
                train_loss = epoch_train_loss / len(train_loader.sampler)
                val_loss = epoch_val_loss / len(val_loader.sampler)

                self.history['train_loss'].append(train_loss)
                self.history['val_loss'].append(val_loss)


                # train & validation error after every epoch
                print("Epoch: {}/{}, Training Loss: {:.4f}, Validation Loss: {:.4f}".format(
                                epoch, self.epochs, train_loss, val_loss))
            # after cross validation
            print(f'Finished training & validation the model for CV..... {cv_index} .........')
            avg_train_loss_after_CV = np.mean(self.history['train_loss'])
            avg_val_loss_after_CV = np.mean(self.history['val_loss'])

            print("Average Training Loss: {:.4f} \t Average Val Loss: {:.4f}".format(avg_train_loss_after_CV, avg_val_loss_after_CV))

        # model testing
        print('Testing the model...')
        test_loss, test_accuracy = self.test(test_loader=test_loader, model=self.model, loss_func=self.loss_func, device=self.device)
        test_loss_ = test_loss / len(test_loader.sampler)

        print("Test Loss: {:.4f}".format(test_loss_))
        print('Finished training & testing the model.')

In [ ]:
batch_size = 32
train_loader = torch.utils.data.DataLoader(TrainData, batch_size=batch_size, shuffle=True, pin_memory=True)
test_loader = torch.utils.data.DataLoader(TestData, batch_size=batch_size, shuffle=True, pin_memory=True)
# model
model = ModelTrainer(lr=4e-06)
model.execute_run(train_loader=train_loader, test_loader=test_loader, fold=3, batch_size=batch_size)

pytorch_model.bin:   0%|          | 0.00/336M [00:00<?, ?B/s]

Training on: cuda
CV: 0
Epoch: 0/10, Training Loss: 4.7869, Validation Loss: 5.7473
Epoch: 1/10, Training Loss: 4.5977, Validation Loss: 4.5755
Epoch: 2/10, Training Loss: 4.5165, Validation Loss: 4.9491
Epoch: 3/10, Training Loss: 4.5116, Validation Loss: 4.5795
Epoch: 4/10, Training Loss: 4.5166, Validation Loss: 4.5759
Epoch: 5/10, Training Loss: 4.5035, Validation Loss: 4.5880
Epoch: 6/10, Training Loss: 4.4676, Validation Loss: 4.7379
Epoch: 7/10, Training Loss: 4.4411, Validation Loss: 4.7397
Epoch: 8/10, Training Loss: 4.4335, Validation Loss: 4.6140
Epoch: 9/10, Training Loss: 4.4261, Validation Loss: 4.9546
Epoch: 10/10, Training Loss: 4.4243, Validation Loss: 4.5761
Finished training & validation the model for CV..... 0 .........
Average Training Loss: 4.5114 	 Average Val Loss: 4.7852
CV: 1
Epoch: 0/10, Training Loss: 4.5209, Validation Loss: 4.4376
Epoch: 1/10, Training Loss: 4.5036, Validation Loss: 4.6988
Epoch: 2/10, Training Loss: 4.5234, Validation Loss: 4.4616
Epoch: 

No CV

In [9]:
import numpy as np
import pandas as pd

class SmilesData2:
    def __init__(self, data):
        self.data = data

    def get_split(self, train_ratio=0.8, val_ratio =0.1, seed=None):
        n = len(self.data)
        indices = np.arange(n)
        if seed is not None:
            np.random.seed(seed)
        np.random.shuffle(indices)
        train_size = int(train_ratio * n)
        val_size = int(val_ratio * n)
        train_indices = indices[:train_size]
        val_indices = indices[train_size:train_size+val_size]
        test_indices = indices[train_size+val_size:]
        train_data = self.data.iloc[train_indices].reset_index(drop=True)
        val_data = self.data.iloc[val_indices].reset_index(drop=True)
        test_data = self.data.iloc[test_indices].reset_index(drop=True)
        return train_data, val_data, test_data

train_data, val_data, test_data = SmilesData2(data).get_split(seed=2024)
print(f'Shape of train data: {train_data.shape}')
print(f'Shape of val data: {val_data.shape}')
print(f'Shape of test data: {test_data.shape}')

Shape of train data: (16696, 4)
Shape of val data: (2087, 4)
Shape of test data: (2087, 4)


In [10]:
# create the train and test datasets
TrainData = Input(train_data, tokenizer)
ValData = Input(val_data, tokenizer)
TestData = Input(test_data, tokenizer)
# Shape
print(f'Length of train data: {len(TrainData)}')
print(f'Length of val data: {len(ValData)}')
print(f'Length of test data: {len(TestData)}')

Length of train data: 16696
Length of val data: 2087
Length of test data: 2087


In [11]:
import numpy as np
from sklearn.model_selection import KFold
from sklearn.metrics import r2_score

import torch
import torch.nn as nn
from torch.utils.data import DataLoader

class Trainer(nn.Module):

    """
        ************** Train/Test the model using cross validation **************
        seed: seed for random number generator
        epochs: number of epochs to train
        lr: learning rate
        train: flag whether to train the model
        log_interval: how many batches to wait before logging training status
        model: takes input_ids: str, attention_mask: str, classification: bool

    """

    def __init__(self, seed = 2023, lr=1e-4, epochs=10):
        super(Trainer, self).__init__()
        self.seed = seed
        self.epochs = epochs
        self.lr = lr
        self.device = torch.device("cuda" if torch.cuda.is_available() else ("mps" if torch.backends.mps.is_available() else "cpu"))
        self.model = SmilesTransformerModel().to(self.device)

        self.optimizer = torch.optim.AdamW(self.model.parameters(), lr=self.lr)

        self.loss_func = nn.MSELoss()

    def evaluate(self, dataloader, model, device, loss_func):
        """Evaluate the network on the test/validation set."""

        eval_loss = 0
        all_labels, all_preds = [], []

        model.eval()
        with torch.no_grad():

            for data in dataloader:
                # get the inputs
                input_ids = data['input_ids'].to(device)
                input_mask = data['attention_mask'].to(device)
                labels = data['label'].to(device)
                # forward pass
                outputs = model(input_ids=input_ids, attention_mask=input_mask)
                # loss and accuracy
                loss = loss_func(outputs, labels)
                eval_loss += loss.sum().item() * input_ids.size(0)
                # accuracy
                all_preds.extend(outputs.detach().cpu().numpy())
                all_labels.extend(labels.detach().cpu().numpy())
        # R-squared
        accuracy = r2_score(all_labels, all_preds)

        return eval_loss, accuracy


    def train(self, model, train_loader, loss_func, optimizer, device):
        """Train the network on the training set."""
        train_loss = 0
        all_labels, all_preds = [], []

        model.train()
        scheduler = torch.optim.lr_scheduler.OneCycleLR(optimizer, max_lr=0.01, steps_per_epoch=len(train_loader), epochs=self.epochs, anneal_strategy='linear')

        for data in train_loader:

            # get the inputs
            input_ids = data['input_ids'].to(device)   # amino acid index numbers
            input_mask = data['attention_mask'].to(device) # attention mask (1 for non-padding token and 0 for padding)
            labels = data['label'].to(device) # True for classification task
            # forward pass
            outputs = self.model(input_ids = input_ids, attention_mask = input_mask)
            # loss and backward pass
            loss = loss_func(input=outputs, target=labels)
            loss.backward()
            optimizer.step()
            scheduler.step()
            optimizer.zero_grad()
            # loss and accuracy
            train_loss += loss.sum().item() * input_ids.size(0)
            # accuracy
            all_preds.extend(outputs.detach().cpu().numpy())
            all_labels.extend(labels.detach().cpu().numpy())
        # R-squared
        train_accuracy = r2_score(all_labels, all_preds)

        return train_loss, train_accuracy

    def execute_run(self, train_loader, test_loader, val_loader, batch_size = 32):
        '''Train, Test and Validate the network on the training set using cross validation.'''

        print(f"Training on: {self.device}")

        torch.manual_seed(self.seed) # set the seed for generating random numbers

        if torch.cuda.is_available():
            torch.cuda.manual_seed(self.seed)

        self.history = {'train_loss': [], 'val_loss': []}

        for epoch in range(0, self.epochs + 1):
            # Training
            epoch_train_loss, epoch_train_accuracy = self.train(model=self.model,
                                                                    train_loader=train_loader, loss_func=self.loss_func,
                                                                    optimizer=self.optimizer, device=self.device)
            # Validation
            epoch_val_loss, epoch_val_accuracy = self.evaluate(dataloader=val_loader,
                                                                    model=self.model, loss_func=self.loss_func,
                                                                    device=self.device)
            #
            train_loss = epoch_train_loss / len(train_loader.sampler)
            val_loss = epoch_val_loss / len(val_loader.sampler)

            self.history['train_loss'].append(train_loss)
            self.history['val_loss'].append(val_loss)


            # train & validation error after every epoch
            print("Epoch: {}/{}, Training Loss: {:.4f}, Validation Loss: {:.4f}, Training Acc: {:.4f}, Validation Accu: {:.4f}".format(
                                epoch, self.epochs, train_loss, val_loss, epoch_train_accuracy, epoch_val_accuracy))
        # after cross validation
        print(f'Finished training & validation the model .........')
        avg_train_loss_after_training = np.mean(self.history['train_loss'])
        avg_val_loss_after_training = np.mean(self.history['val_loss'])

        print("Average Training Loss: {:.4f} \t Average Val Loss: {:.4f}".format(avg_train_loss_after_training, avg_val_loss_after_training))

        # model testing
        print('Testing the model...')
        test_loss, test_accuracy = self.evaluate(dataloader=test_loader, model=self.model, loss_func=self.loss_func, device=self.device)
        test_loss_ = test_loss / len(test_loader.sampler)

        print("Test Loss: {:.4f}, Test Accuracy: {:.4f}".format(test_loss_, test_accuracy))
        print('Finished training & testing the model.')

In [ ]:
batch_size = 32
train_loader = torch.utils.data.DataLoader(TrainData, batch_size=batch_size, shuffle=True, pin_memory=True)
valid_loader = torch.utils.data.DataLoader(ValData, batch_size=batch_size, shuffle=True, pin_memory=True)
test_loader = torch.utils.data.DataLoader(TestData, batch_size=batch_size, shuffle=True, pin_memory=True)
# model
model2 = Trainer(lr=1e-04)
model2.execute_run(train_loader=train_loader, val_loader=valid_loader, test_loader=test_loader, batch_size=batch_size)

pytorch_model.bin:   0%|          | 0.00/336M [00:00<?, ?B/s]

Training on: cuda
Epoch: 0/10, Training Loss: 4.7089, Validation Loss: 4.6585, Training Acc: -0.0622, Validation Accu: -0.0144
Epoch: 1/10, Training Loss: 4.5894, Validation Loss: 4.7846, Training Acc: -0.0353, Validation Accu: -0.0419
Epoch: 2/10, Training Loss: 4.6769, Validation Loss: 4.7725, Training Acc: -0.0550, Validation Accu: -0.0393
Epoch: 3/10, Training Loss: 4.5968, Validation Loss: 5.1214, Training Acc: -0.0369, Validation Accu: -0.1152
Epoch: 4/10, Training Loss: 4.5661, Validation Loss: 4.5933, Training Acc: -0.0300, Validation Accu: -0.0002
Epoch: 5/10, Training Loss: 4.5078, Validation Loss: 4.6217, Training Acc: -0.0169, Validation Accu: -0.0064
Epoch: 6/10, Training Loss: 4.4947, Validation Loss: 4.6548, Training Acc: -0.0139, Validation Accu: -0.0136
Epoch: 7/10, Training Loss: 4.4822, Validation Loss: 4.6168, Training Acc: -0.0111, Validation Accu: -0.0054
Epoch: 8/10, Training Loss: 4.4905, Validation Loss: 4.6438, Training Acc: -0.0130, Validation Accu: -0.0112
